In [1]:
import pandas as pd
import math
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import time

In [2]:
import os
print(os.getcwd())

C:\Users\Charlie\Documents\portfolio_projects


In [3]:
#import sys
#sys.path.insert(0,"C:\\Users\Charles\Documents\GitHub\DSCI-633\\assignments")

In [5]:
from my_evaluation import my_evaluation
from my_GA import my_GA

def obj_func1(predictions, actuals, pred_proba=None):
    # Two objectives: higher recall and lower false positive rate
    eval = my_evaluation(predictions, actuals, pred_proba)
    return [eval.recall(), - eval.precision()]

def obj_func2(predictions, actuals, pred_proba=None):
    # Two objectives: higher recall and lower false positive rate
    eval = my_evaluation(predictions, actuals, pred_proba)
    return eval.f1()

In [6]:
#Model

In [7]:
class my_model():
    def fit(self, X, y):
        # do not exceed 29 mins
        X = X.drop(['telecommuting','has_company_logo','has_questions'],axis=1)
        self.preprocessor = TfidfVectorizer(stop_words='english', norm='l2', use_idf=True, smooth_idf=False,lowercase=False,max_df=.432,min_df=0.0)
        XX = self.preprocessor.fit_transform(X["description"].values.astype('U'))

        self.clf = SGDClassifier(class_weight="balanced",alpha=0.00005,average=False,power_t=4.15)
        self.clf.fit(XX, y)
        return

    def predict(self, X):
        # remember to apply the same preprocessing in fit() on test data before making predictions
        X = X.drop(['telecommuting','has_company_logo','has_questions'],axis=1)
        XX = self.preprocessor.transform(X["description"].values.astype('U'))
        predictions = self.clf.predict(XX)
        return predictions

In [8]:
#Functions

In [9]:
def test(data):#maxFeatures):#,maxDF,minDF):
    y = data["fraudulent"]
    X = data.drop(['fraudulent'], axis=1)
    split_point = int(0.8 * len(y))
    #train = data[:split_point]
    split_point = int(0.8 * len(y))
    #X_train, y_train = getTrainingData(train)
    X_train = X.iloc[:split_point]
    X_test = X.iloc[split_point:]
    y_train = y.iloc[:split_point]
    y_test = y.iloc[split_point:]
    clf = my_model()
    clf.fit(X_train,y_train)
    predictions = clf.predict(X_test)
    eval = my_evaluation(predictions, y_test)
    f1 = eval.f1(target=1)
    return f1

In [10]:
def tuneDF(X,rangeMaxDF,rangeMinDF,rangeMaxFeatures,incrementFactor):#works for a multiple values, not optimal since needs editing in other methods, do this in HPTuning.py
    incrementMaxDF = (rangeMaxDF[1] - rangeMaxDF[0]) / incrementFactor
    incrementMinDF = (rangeMinDF[1] - rangeMinDF[0]) / incrementFactor
    bestF1 = 0
    bestValues = [-1,-1]
    print(incrementMaxDF)
    print(incrementMinDF)
    for i in range(incrementFactor):
        for j in range(incrementFactor):
            locMax = incrementMaxDF * i
            locMin = incrementMinDF * j
            try:
                newF1 = test(data,locMax,locMin)
                if newF1 >= bestF1:
                    bestF1 = newF1
                    bestValues = [incrementMaxDF * i, incrementMinDF * j]
                print("no exc")
            except:
                print("exc handled")
    return [bestValues,bestF1]

In [11]:
def tuneOther(X,rangeMaxFeatures,incrementFactor):#works for a single value, not optimal since needs editing in other methods, do this in HPTuning.py
    incrementMaxFeatures = (rangeMaxFeatures[1] - rangeMaxFeatures[0]) / incrementFactor
    bestF1 = 0
    bestValues = [1]
    print(incrementMaxFeatures)
    count = 0
    for i in range(incrementFactor):#for maxfeatures
        if i == 0:
            locMaxFeatures = 1
        else:
            locMaxFeatures = incrementMaxFeatures * i
        print(locMaxFeatures)
        newF1 = test(data,int(locMaxFeatures))
        if newF1 >= bestF1:
                bestF1 = newF1
                bestValues = [locMaxFeatures]
        count += 1
                        
    #print("COUNT")
    #print(count)
    return [bestValues,bestF1]

In [13]:
if __name__ == "__main__":
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!        I'm getting way gd high scores since I balanced the training data, seems wrong so look into that        !!!!!!!!!!!!!!!!!!!!!!!!
    start = time.time()
    # Load data
    data = pd.read_csv("job_train.csv")
    # Replace missing values with empty strings
    data = data.fillna("")
    """#For single test
    f1 = test(data)#,bestVals[0])#,maxDF,minDF)
    print("F1 score: %f" % f1)
    #print("F1Sklearn: " + str(f1[1]))
    runtime = (time.time() - start) / 60.0
    print(runtime)
    #print(bestVals)
    #print(maxDF)
    #print(minDF)
    ##BEST MAX VAL IS .423
    ##Best MIN VAL IS 0.0
    ##BEST MAXFEATURES IS None
    """
    #For testing average over cnt runs
    sumf1 = 0
    cnt = 10
    for i in range(cnt):
        sumf1 += test(data)
    print("f1: " + str(sumf1/cnt))
    runtime = (time.time() - start) / 60.0
    print(str(runtime/cnt))

f1: 0.8561665811345204
0.02506255785624186


In [ ]:
#Now I need to make a little application where you can use the trained model to filter out bad job listings, maybe look more
#into how many good offers it might accidentally misclassify as bad job offers, because if thats high there is no point.